<a href="https://colab.research.google.com/github/Tom-Jung/ML_Colab/blob/main/TF_2_x_LEC_31_Brain_Tumor_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dropout, Dense, GlobalAveragePooling2D 
from tensorflow.keras.applications import MobileNet 
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

root dir
data root dir
train data root dir
test data root dir

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [2]:
import os

ROOT_DIR = '/content'

DATA_ROOT_DIR = os.path.join(ROOT_DIR, 'brain_tumor')

TRAIN_DATA_ROOT_DIR = os.path.join(DATA_ROOT_DIR, 'Training')

TEST_DATA_ROOT_DIR = os.path.join(DATA_ROOT_DIR, 'Testing')

In [5]:
import shutil
import os

try:
    dataset_path = '/content/gdrive/My Drive/Colab Notebooks/dataset'
    
    shutil.copy(os.path.join(dataset_path, 'brain_tumors.zip'), '/content')

except Exception as err:
    print(str(err))

In [6]:
import os
import shutil

if os.path.exists(DATA_ROOT_DIR):
    
    shutil.rmtree(DATA_ROOT_DIR)
    
    print(DATA_ROOT_DIR + ' is removed !!!')

In [8]:
# 압축파일 풀기

import zipfile

with zipfile.ZipFile(os.path.join(ROOT_DIR, 'brain_tumors.zip'), 'r') as target_file:

    target_file.extractall(DATA_ROOT_DIR) 

In [9]:
import os

#========================================================
# train 정답 및 전체 데이터 개수 확인
#========================================================

train_label_name_list = os.listdir(TRAIN_DATA_ROOT_DIR)

print(train_label_name_list)

for label_name in train_label_name_list:

    print('train label : ', label_name,' => ', len(os.listdir(os.path.join(TRAIN_DATA_ROOT_DIR, label_name))))

print('=====================================================')

#========================================================
# test 정답 및 전체 데이터 개수 확인
#========================================================

test_label_name_list = os.listdir(TEST_DATA_ROOT_DIR)

print(test_label_name_list)

for label_name in test_label_name_list:

    print('test label : ', label_name, ' => ', len(os.listdir(os.path.join(TEST_DATA_ROOT_DIR, label_name))))

print('=====================================================')
['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
train label :  glioma_tumor  =>  826
train label :  meningioma_tumor  =>  822
train label :  no_tumor  =>  395
train label :  pituitary_tumor  =>  827
=====================================================
['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
test label :  glioma_tumor  =>  100
test label :  meningioma_tumor  =>  115
test label :  no_tumor  =>  105
test label :  pituitary_tumor  =>  74
=====================================================


SyntaxError: ignored